In [ ]:
import random
import unittest


def roll_dice():
    """Simulates rolling five dice and returns a list of their values."""
    dice_roll = []
    for _ in range(5):
        dice_roll.append(random.randint(1, 6))
    return dice_roll


def calculate_score(dice_roll, category):
    """Calculates the score for a given category and dice roll."""
    scores = {
        "Aces": sum(die == 1 for die in dice_roll),
        "Twos": sum(die == 2 for die in dice_roll),
        "Threes": sum(die == 3 for die in dice_roll),
        "Fours": sum(die == 4 for die in dice_roll),
        "Fives": sum(die == 5 for die in dice_roll),
        "Sixes": sum(die == 6 for die in dice_roll),
        "Three of a Kind": max(dice_roll) * 3 if any(dice_roll.count(die) >= 3 for die in dice_roll) else 0,
        "Four of a Kind": max(dice_roll) * 4 if any(dice_roll.count(die) >= 4 for die in dice_roll) else 0,
        "Full House": 25 if (any(dice_roll.count(die) == 3 for die in dice_roll) and
                            any(dice_roll.count(die) == 2 for die in dice_roll)) else 0,
        "Small Straight": 30 if sorted(dice_roll) == list(range(min(dice_roll), min(dice_roll) + 4)) else 0,
        "Large Straight": 40 if sorted(dice_roll) == list(range(1, 6)) else 0,
        "Yahtzee": 50 if all(die == dice_roll[0] for die in dice_roll[1:]) else 0,
    }
    return scores.get(category, 0)  # Return 0 for invalid category


def is_category_available(scorecard, category):
    """Checks if a category is available (not scored yet) in the scorecard."""
    return scorecard.get(category, 0) == 0


def update_scorecard(scorecard, category, score):
    """Updates the scorecard with the score for a given category."""
    scorecard[category] = score


def display_scorecard(scorecard):
    """Prints the Yahtzee scorecard."""
    print("Upper Section:")
    for category in ("Aces", "Twos", "Threes", "Fours", "Fives", "Sixes"):
        print(f"{category}: {scorecard.get(category, 0)}")
    print("Bonus:", scorecard.get("Bonus", 0))
    print("Total Upper Section:", sum(scorecard.get(cat, 0) for cat in ("Aces", "Twos", "Threes", "Fours", "Fives", "Sixes")))
    print("Lower Section:")
    for category in ("Three of a Kind", "Four of a Kind", "Full House", "Small Straight", "Large Straight", "Yahtzee"):
        print(f"{category}: {scorecard.get(category, 0)}")
    print("Yahtzee Bonus:", scorecard.get("Yahtzee Bonus", 0))
    print(f"Total Score: {sum(scorecard.values())}")
def play_yahtzee(dice_roll=None, re_roll=[], category=None):
    """Simulates a single game round of Yahtzee."""
    scorecard = {
        "Aces": 0, "Twos": 0, "Threes": 0, "Fours": 0, "Fives": 0, "Sixes": 0,
        "Bonus": 0, "Three of a Kind": 0, "Four of a Kind": 0, "Full House": 0,
        "Small Straight": 0, "Large Straight": 0, "Yahtzee": 0, "Yahtzee Bonus": 0
    }

    if dice_roll is None:
        dice_roll = roll_dice()
    print(f"Dice Roll: {dice_roll}")
    num_rolls = 0
    while num_rolls < 3:
        re_roll_input = input("Enter dice numbers to re-roll (e.g., 1 3 5): ")
        try:
            re_roll = [int(die) for die in re_roll_input.split()]
        except ValueError:
            print("Invalid input. Please enter numbers between 1 and 5 separated by spaces.")
            continue
        if not all(1 <= die <= 5 for die in re_roll):
            print("Invalid dice numbers. Please enter numbers between 1 and 5.")
            continue
        re_roll_dice = [die for die in dice_roll if int(die) not in (re_roll)]
        for i in range(len(re_roll_dice)):
            re_roll_dice[i] = roll_dice()[0]  # Simulate re-rolling a single die
        dice_roll = re_roll_dice + [die for die in dice_roll if int(die) in (re_roll)]
        num_rolls += 1
        print(f"Re-rolled Dice: {dice_roll}")

        available_categories = [cat for cat in scorecard.keys() if is_category_available(scorecard, cat)]
        print("\nAvailable Categories:")
        for category in available_categories:
            print(category)

        # Choose category (consider adding choose_best_category function here)
        if category is None:
            while category not in available_categories:
                category = input("Choose a category to score in: ")
                if category not in available_categories:
                    print("Invalid category. Choose an available category: ")

        score = calculate_score(dice_roll, category)
        update_scorecard(scorecard, category, score)

        # Upper section bonus logic
        if sum(scorecard.get(cat, 0) for cat in ("Aces", "Twos", "Threes", "Fours", "Fives", "Sixes")) >= 63:
            scorecard["Bonus"] = 35

        # Yahtzee bonus logic
        if scorecard["Yahtzee"] > 1:
            scorecard["Yahtzee Bonus"] += 100

        print(f"\nScored {score} in {category}.")
        display_scorecard(scorecard)


In [ ]:
import unittest


def test_simple_upper_section(self):
    """Tests scoring in the upper section (Aces, Twos, etc.)."""
    dice_roll = [3, 3, 3, 1, 5]
    expected_scorecard = {
        "Aces": 0, "Twos": 0, "Threes": 9, "Fours": 0, "Fives": 0, "Sixes": 0,
        "Bonus": 0, "Three of a Kind": 0, "Four of a Kind": 0, "Full House": 0,
        "Small Straight": 0, "Large Straight": 0, "Yahtzee": 0, "Yahtzee Bonus": 0
    }
    scorecard = play_yahtzee(dice_roll)
    self.assertEqual(scorecard["Threes"], 9)
    self.assertDictEqual(scorecard, expected_scorecard)


def test_re_roll_and_lower_section(self):
    """Tests re-rolling dice and scoring in the lower section."""
    dice_roll = [2, 2, 2, 4, 1]
    re_roll = [5]  # Simulate user re-rolling the 4
    expected_scorecard = {
        "Aces": 0, "Twos": 6, "Threes": 0, "Fours": 0, "Fives": 0, "Sixes": 0,
        "Bonus": 0, "Three of a Kind": 0, "Four of a Kind": 0, "Full House": 25,
        "Small Straight": 0, "Large Straight": 0, "Yahtzee": 0, "Yahtzee Bonus": 0
    }
    scorecard = play_yahtzee(dice_roll, re_roll)
    self.assertEqual(scorecard["Full House"], 25)
    self.assertDictEqual(scorecard, expected_scorecard)


def test_yahtzee_and_bonus(self):
    """Tests scoring Yahtzee and Yahtzee bonus."""
    dice_roll = [5, 5, 5, 5, 5]
    expected_scorecard = {
        "Aces": 0, "Twos": 0, "Threes": 0, "Fours": 0, "Fives": 50, "Sixes": 0,
        "Bonus": 35, "Three of a Kind": 0, "Four of a Kind": 0, "Full House": 0,
        "Small Straight": 0, "Large Straight": 0, "Yahtzee": 50, "Yahtzee Bonus": 0
    }
    scorecard = play_yahtzee(dice_roll)
    self.assertEqual(scorecard["Yahtzee"], 50)
    self.assertDictEqual(scorecard, expected_scorecard)


def test_invalid_re_roll(self):
    """Tests handling of invalid re-roll input (non-numeric characters)."""
    dice_roll = [1, 4, 3, 2, 6]
    with self.assertRaises(ValueError):
        play_yahtzee(dice_roll, re_roll="abc")  # Simulate invalid input


def test_unavailable_category(self):
    """Tests handling of choosing an already scored category."""
    dice_roll = [2, 4, 1, 3, 5]
    # Simulate user choosing an already scored category (Twos)
    with self.assertRaises(ValueError):
        play_yahtzee(dice_roll, re_roll=[], category="Twos")


class YahtzeeTest(unittest.TestCase):
    def test_all(self):
        """Runs all the test cases."""
        test_simple_upper_section(self)
        test_re_roll_and_lower_section(self)
        test_yahtzee_and_bonus(self)
        test_invalid_re_roll(self)
        test_unavailable_category(self)


if __name__ == "__main__":
    unittest.main()


E
ERROR: /root/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/root/'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


SystemExit: True

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
